In [2]:
import cv2
import numpy as np

In [3]:
# 计算梯度幅值和方向
def Gradient_1(image):
    Size = np.shape(image)
    P = np.zeros(Size)
    Q = np.zeros(Size)
    M = np.zeros(Size)
    theta = np.zeros(Size)
    image = np.float16(image)
    for i in range(0, Size[0] - 1):
        for j in range(0, Size[1] - 1):
            P[i, j] = image[i + 1, j]  - image[i, j]  + image[
                i + 1, j + 1]  - image[i, j + 1] 
            Q[i, j] = image[i, j + 1]  - image[i, j]  + image[
                i + 1, j + 1]  - image[i + 1, j]

            M[i, j] = np.sqrt(P[i, j]**2 + Q[i, j]**2)
            if (P[i, j] != 0):
                theta[i, j] = np.arctan(Q[i, j] / P[i, j])
            else:
                theta[i, j] = np.pi / 2
    return np.uint8(M), theta


def Gradient_Sobel(image):
    Size = np.shape(image)
    P = np.zeros(Size)
    Q = np.zeros(Size)
    M = np.zeros(Size)
    theta = np.zeros(Size)
    image = np.float16(image)
    for i in range(1, Size[0] - 1):
        for j in range(1, Size[1] - 1):
            P[i, j] = 2 * (image[i + 1, j] - image[i - 1, j]) + image[
                i + 1, j + 1] - image[i - 1, j + 1] + image[i + 1, j -
                                                            1] - image[i - 1,
                                                                       j - 1]
            Q[i, j] = 2 * (image[i, j + 1] - image[i, j - 1]) + image[
                i + 1, j + 1] - image[i + 1, j - 1]+ image[i - 1, j +
                                                            1] - image[i - 1,
                                                                       j - 1]

            M[i, j] = np.sqrt(P[i, j]**2 + Q[i, j]**2)
            if (P[i, j] != 0):
                theta[i, j] = np.arctan(Q[i, j] / P[i, j])
            else:
                theta[i, j] = np.pi / 2
    return np.uint8(M), theta

In [4]:
# 方向角离散化
def AngleDiscretization(theta):
    Size = np.shape(theta)
    theta2 = np.zeros(Size)
    for i in range(1, Size[0] - 1):
        for j in range(1, Size[1] - 1):
            if (theta[i, j] >= -np.pi / 8 and theta[i, j] < np.pi / 8):
                theta2[i, j] = 0
            elif (theta[i, j] >= np.pi / 8 and theta[i, j] < 3 * np.pi / 8):
                theta2[i, j] = 1
            elif (theta[i, j] >= 3 * np.pi / 8
                  or theta[i, j] < -3 * np.pi / 8):
                theta2[i, j] = 2
            else:
                theta2[i, j] = 3
    return theta2

# 非极大值抑制
def NonMaxSuppression(M, theta2):
    Size = np.shape(M)
    temp = np.zeros(Size) + 1
    for i in range(1, Size[0] - 1):
        for j in range(1, Size[1] - 1):
            if (theta2[i, j] == 0):
                if (M[i, j] < M[i + 1, j] or M[i, j] < M[i - 1, j]):
                    temp[i, j] = 0
            elif (theta2[i, j] == 1):
                if (M[i, j] < M[i + 1, j + 1] or M[i, j] < M[i - 1, j - 1]):
                    temp[i, j] = 0
            elif (theta2[i, j] == 2):
                if (M[i, j] < M[i, j + 1] or M[i, j] < M[i, j - 1]):
                    temp[i, j] = 0
            else:
                if (M[i, j] < M[i - 1, j + 1] or M[i, j] < M[i + 1, j - 1]):
                    temp[i, j] = 0
    return np.uint8(temp * M)


In [5]:
# 双阈值连接边缘
def EdgeJoint(M, highThreshold,lowThreshold):
    Size = np.shape(M)
    high = np.zeros(np.shape(M))
    low = np.zeros(np.shape(M))
    for i in range(1,Size[0]-1):
        for j in range(1,Size[1]-1):
            if(M[i,j]>highThreshold):
                high[i,j] = 1
            elif(M[i,j]>lowThreshold):
                low[i,j] = 1
    
    for i in range(1,Size[0]-1):
        for j in range(1,Size[0]-1):
            if(high[i,j]==1 and low[i,j]!=1):
                [high,low]=Joint(high,low,i,j)
            # if(low[i,j]==1):
                # for k in range(0,3):
                #     for l in range(0,3):
                #         if(high[i-1+k,j-1+l]==1):
                #             high[i,j] = 1
                #             i = 0
                #             j = 0
    return np.uint8(high*M)
    
def Joint(high,low,i,j):
    Size = np.shape(high)
    if(i!=0 and j!=0 and i!=Size[0]-1 and j !=Size[0]-1):
        for k in range(0,3):
            for m in range(0,3):
                if(high[i-1+k,j-1+m]!=1 and low[i-1+k,j-1+m]==1):
                    high[i-1+k,j-1+m]=1
                    [high,low]=Joint(high,low,i-1+k,j-1+m)
    return [high,low]

In [6]:
# 边缘检测函数
def EdgeDect(image,highThreshold,lowThreshold):
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imageGauss = cv2.GaussianBlur(imageGray,(3,3),1)
    magGradient, angleGradient= Gradient_Sobel(imageGauss)
    angleDiscretion = AngleDiscretization(angleGradient)
    magSuppression = NonMaxSuppression(magGradient,angleDiscretion)
    magEdgejoint = EdgeJoint(magSuppression,highThreshold,lowThreshold)
    # images = np.hstack([imageGray,magGradient,magSuppression,magEdgejoint])
    # cv2.imshow('figure2',images)
    # k = cv2.waitKey(0)
    # if k== 27:
    #     cv2.destroyAllWindows()
    return [imageGray,magGradient,magSuppression,magEdgejoint]


def mask(maskImage,image):
    try:
        if (np.shape(maskImage)!=np.shape(image)[0:2]):
            raise ValueError('!图片大小不一致!')
    except Exception as e:
        print(e)
        return

    Size = np.shape(image)
    for i in range(0,Size[0]):
        for j in range(0,Size[1]):
            if(maskImage[i,j]==0):
                for k in range(0,3):
                    image[i,j,k] = 0
    return  np.uint8(image)

In [18]:
# 导入图片
image0 = cv2.imread('25741.jpg')
highThreshold = 150
lowThreshold = 10
[imageGray,magGradient,magSuppression,magEdgejoint]=EdgeDect(image0,highThreshold,lowThreshold)
colorImage = mask(magEdgejoint,image0)


In [19]:
images = np.hstack([magGradient,magSuppression,magEdgejoint])
cv2.imshow('figure',colorImage)
k = cv2.waitKey(0)
if k== 27:
    cv2.destroyAllWindows()
cv2.imwrite('magGradient.png', 255-magGradient, [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite('magsuppression.png', 255-magSuppression, [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite('magEdgejoint.png', 255-magEdgejoint, [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite('colorImage.png', colorImage, [cv2.IMWRITE_PNG_COMPRESSION, 0])


True